## Face extraction using visual words

---

In this notebook I will use bag-of-visual generated by the [last notebook](https://colab.research.google.com/drive/1J5B1rTAGaAfFelf8P9d4lXzjjH1j_WBr?usp=sharing) and apply multiple models in order to select the best for bounding box. 

---

references:
  - sklearn for multiregression : [models](https://scikit-learn.org/stable/modules/multiclass.html)
  - save and load models : [post](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [ ]:
!gdown 1O5O20iNpg_aNiNLXc16gIuvUZ3j6S9V0

Downloading...
From: https://drive.google.com/uc?id=1O5O20iNpg_aNiNLXc16gIuvUZ3j6S9V0
To: /content/wider_data_with_visual_words.csv
100% 9.48M/9.48M [00:00<00:00, 115MB/s]


In [ ]:
!mkdir models

In [ ]:
from sklearn.metrics import classification_report,plot_confusion_matrix,confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC 
from sklearn import metrics
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import ast


tqdm.pandas()
sns.set_theme()
MODEL_DIR_NAME = 'models/'

In [ ]:
def validation_metrics(y_pred, y_true, show_img=True):
  rmse = np.sqrt(np.square(y_pred - y_true).mean(axis=-1))
  fig = px.histogram(rmse)
  fig.show()
  return np.mean(rmse)

In [ ]:
df = pd.read_csv('wider_data_with_visual_words.csv')
df['visual_words_histogram'] = [np.array(ast.literal_eval(cur_vword)) for cur_vword in df['visual_words_histogram']]
df.head(3)

,img_location,x0,y0,x1,y2,visual_words_histogram
0,wider_data_resized/0_Parade_marchingband_1_849...,210.46875,114.368230,267.65625,166.00722,"[4, 1, 4, 1, 1, 0, 5, 4, 1, 2, 10, 1, 2, 7, 5,..."
1,wider_data_resized/0_Parade_Parade_0_904.jpg,169.21875,32.849160,292.50000,146.48044,"[11, 7, 3, 3, 6, 2, 2, 9, 0, 4, 1, 5, 12, 0, 8..."
2,wider_data_resized/0_Parade_Parade_0_577.jpg,137.81250,90.966515,351.09375,267.44156,"[6, 4, 6, 8, 1, 5, 5, 1, 2, 3, 7, 1, 1, 2, 8, ..."


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
len(df_train), len(df_test)

(4029, 1728)

In [ ]:
x_train = np.array(list(df_train['visual_words_histogram'].values))
y_train = np.array(list(df_train[['x0', 'y0', 'x1', 'y2']].values))

x_test = np.array(list(df_test['visual_words_histogram'].values))
y_test = np.array(list(df_test[['x0', 'y0', 'x1', 'y2']].values))

In [ ]:
def train_model_and_test_sk_model(x_train, y_train, x_test_, y_test, sk_model, filename):
  sk_model.fit(x_train, y_train)
  y_pred = sk_model.predict(x_test)
  pickle.dump(sk_model, open(MODEL_DIR_NAME + filename, 'wb'))

  print("rsme : ", validation_metrics(y_test, y_pred))


## Defining the models for bounding box generation

---

Now we define a model predict the bounding boxes of the image based on bag of visual words. 

So first we can try some base models :

 - GradientBoostingRegressor
 - SGDRegressor
 - KNNr
 - simple NN


In order to check the model performance we generate the histogram of RMSE of each example, them we display the mean of the RMSE distribution. The perfect model will have a mean RMSE equal to zero and a histogram concentrated around zero.

#### SKlearn gradient boosting

---

In [ ]:
sk_model = MultiOutputRegressor(GradientBoostingRegressor(random_state=42, verbose=1))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test,
                              sk_model, filename='gradient_model.sav')

      Iter       Train Loss   Remaining Time 
         1        5389.1963           17.55s
         2        5345.6859           17.27s
         3        5300.4304           17.06s
         4        5261.6084           16.87s
         5        5224.0612           16.65s
         6        5189.7779           16.53s
         7        5157.1185           16.31s
         8        5122.0807           16.12s
         9        5089.7621           16.14s
        10        5061.3733           15.93s
        20        4806.8909           14.01s
        30        4602.4566           12.16s
        40        4437.2607           10.39s
        50        4306.8994            8.64s
        60        4179.4917            6.90s
        70        4078.7711            5.47s
        80        3944.6623            3.87s
        90        3852.3083            1.91s
       100        3711.9413            0.00s
      Iter       Train Loss   Remaining Time 
         1        4435.7070           16.52s
        

rsme :  64.50013127858352


### SVG for regression

---

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
sk_model = MultiOutputRegressor(SGDRegressor(random_state=42, verbose=1))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test, sk_model, 'SGDRegressor.sav')

-- Epoch 1
Norm: 2746730470584.48, NNZs: 512, Bias: 5940779846.314019, T: 4029, Avg. loss: 1870700406753390288242212864.000000
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 2749685587471.62, NNZs: 512, Bias: 5228702825.164730, T: 8058, Avg. loss: 1057987326745522561106313216.000000
Total training time: 0.02 seconds.
-- Epoch 3
Norm: 1446385267646.15, NNZs: 512, Bias: 45390637411.571205, T: 12087, Avg. loss: 3048085878289269277534978048.000000
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 1568435692039.82, NNZs: 512, Bias: 35539592989.624420, T: 16116, Avg. loss: 400197007699625461610971136.000000
Total training time: 0.04 seconds.
-- Epoch 5
Norm: 1308823711871.48, NNZs: 512, Bias: 61366745659.502228, T: 20145, Avg. loss: 431355812017752179078070272.000000
Total training time: 0.04 seconds.
-- Epoch 6
Norm: 1285959975576.41, NNZs: 512, Bias: 88596650522.395218, T: 24174, Avg. loss: 1057215865734192583055245312.000000
Total training time: 0.05 seconds.
-- Epoch 7
Norm: 16722

rsme :  3607877429814.3315


KNN for regression

--- 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
sk_model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=10))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test, sk_model, 'knnRegression.sav')

rsme :  68.15663198223791


Simple NN for regression

--- 

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
sk_model = MultiOutputRegressor(MLPRegressor(random_state=42, verbose=1))
train_model_and_test_sk_model(x_train, y_train, x_test, y_test, sk_model, 'simpleNN.sav')

Iteration 1, loss = 12506.12927210
Iteration 2, loss = 8103.47013512
Iteration 3, loss = 7148.76000550
Iteration 4, loss = 6658.64650524
Iteration 5, loss = 6270.82046050
Iteration 6, loss = 5997.50633269
Iteration 7, loss = 5774.65927832
Iteration 8, loss = 5601.23357453
Iteration 9, loss = 5528.04399885
Iteration 10, loss = 5362.02626393
Iteration 11, loss = 5271.10847352
Iteration 12, loss = 5173.63354077
Iteration 13, loss = 5097.73201247
Iteration 14, loss = 5026.40821767
Iteration 15, loss = 4952.35680630
Iteration 16, loss = 4875.66617707
Iteration 17, loss = 4823.35495967
Iteration 18, loss = 4761.87362941
Iteration 19, loss = 4705.05809147
Iteration 20, loss = 4653.16043857
Iteration 21, loss = 4599.15844033
Iteration 22, loss = 4566.97968244
Iteration 23, loss = 4502.38001354
Iteration 24, loss = 4465.11331373
Iteration 25, loss = 4429.00280348
Iteration 26, loss = 4377.04581521
Iteration 27, loss = 4338.51517988
Iteration 28, loss = 4295.21453108
Iteration 29, loss = 4275.08

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



Iteration 3, loss = 3001.93122455
Iteration 4, loss = 2879.91822545
Iteration 5, loss = 2808.05803399
Iteration 6, loss = 2791.23296576
Iteration 7, loss = 2738.12653845
Iteration 8, loss = 2673.78565702
Iteration 9, loss = 2645.43431252
Iteration 10, loss = 2615.57253573
Iteration 11, loss = 2589.35006919
Iteration 12, loss = 2567.99803416
Iteration 13, loss = 2525.82557456
Iteration 14, loss = 2493.98420199
Iteration 15, loss = 2472.31731359
Iteration 16, loss = 2446.30387582
Iteration 17, loss = 2445.35096668
Iteration 18, loss = 2399.90568340
Iteration 19, loss = 2375.90906786
Iteration 20, loss = 2353.15053364
Iteration 21, loss = 2326.96221555
Iteration 22, loss = 2323.42384441
Iteration 23, loss = 2286.90281301
Iteration 24, loss = 2271.50580152
Iteration 25, loss = 2261.07927295
Iteration 26, loss = 2242.27985348
Iteration 27, loss = 2243.71821813
Iteration 28, loss = 2198.55096161
Iteration 29, loss = 2217.63844751
Iteration 30, loss = 2164.76521429
Iteration 31, loss = 2151.5

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



Iteration 1, loss = 27687.06316063
Iteration 2, loss = 16481.88304415
Iteration 3, loss = 15129.10392326
Iteration 4, loss = 14184.90398789
Iteration 5, loss = 13320.35545092
Iteration 6, loss = 12576.48231911
Iteration 7, loss = 11930.40666914
Iteration 8, loss = 11398.60132826
Iteration 9, loss = 11019.73606369
Iteration 10, loss = 10583.45869141
Iteration 11, loss = 10260.60013564
Iteration 12, loss = 9949.03932738
Iteration 13, loss = 9649.09326700
Iteration 14, loss = 9375.72859470
Iteration 15, loss = 9121.30618773
Iteration 16, loss = 8898.77017503
Iteration 17, loss = 8724.81051016
Iteration 18, loss = 8539.98633417
Iteration 19, loss = 8390.89431330
Iteration 20, loss = 8248.93115719
Iteration 21, loss = 8109.39353521
Iteration 22, loss = 8006.23332465
Iteration 23, loss = 7871.11032042
Iteration 24, loss = 7781.20156477
Iteration 25, loss = 7675.98288286
Iteration 26, loss = 7574.54265926
Iteration 27, loss = 7480.74585903
Iteration 28, loss = 7389.42942040
Iteration 29, loss

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



Iteration 1, loss = 17716.85620941
Iteration 2, loss = 13028.91401225
Iteration 3, loss = 11878.41103964
Iteration 4, loss = 11238.92389834
Iteration 5, loss = 10686.90679655
Iteration 6, loss = 10274.22117028
Iteration 7, loss = 9866.37649112
Iteration 8, loss = 9543.92860789
Iteration 9, loss = 9297.93927558
Iteration 10, loss = 9014.54944804
Iteration 11, loss = 8804.42998152
Iteration 12, loss = 8566.46484158
Iteration 13, loss = 8363.74390747
Iteration 14, loss = 8184.43902454
Iteration 15, loss = 8033.23994507
Iteration 16, loss = 7871.18924001
Iteration 17, loss = 7731.66459104
Iteration 18, loss = 7601.55083507
Iteration 19, loss = 7472.10298155
Iteration 20, loss = 7376.43962986
Iteration 21, loss = 7259.71876592
Iteration 22, loss = 7175.23732790
Iteration 23, loss = 7044.08641174
Iteration 24, loss = 6970.92312403
Iteration 25, loss = 6889.13341092
Iteration 26, loss = 6776.90736851
Iteration 27, loss = 6710.53238623
Iteration 28, loss = 6629.07937503
Iteration 29, loss = 65

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



rsme :  88.70225759099223


In [ ]:
df_final_results = {'gradientBosting' : [64.50],
                    'knnRegression': [68.15],
                    'SVGRegression': [3607877429814.33] ,
                    'simpleNN': [88.70]}

In [ ]:
pd.DataFrame(df_final_results)

,gradientBosting,knnRegression,SVGRegression,simpleNN
0,64.5,68.15,3.607877e+12,88.7


In [ ]:
!zip -r models_for_face_extraction.zip models

  adding: models/ (stored 0%)
  adding: models/SGDRegressor.sav (deflated 7%)
  adding: models/knnRegression.sav (deflated 91%)
  adding: models/simpleNN.sav (deflated 4%)
  adding: models/gradient_model.sav (deflated 65%)
